In [1]:
import pandas as pd
import numpy as np

In [2]:
dfoff = pd.read_csv('./data/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('./data/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('./data/ccf_online_stage1_train.csv')

dfsub = pd.DataFrame({'User_id':'用户ID',
                      'Coupon_id':'优惠券ID',
                      'Date_received':'领取优惠券日期',
                      'Probability':'预测值'},index=[1])

# 只用线下数据，不做特征，找几个基本的模型，然后提交

## 1、预处理
正负样本标签(优惠券消费1，普通消费0，领券不消费2)  
训练和测试子集划分  
还是要处理一下特征，优惠力度、消费日期等  

In [35]:
#  训练集
def get_label(x):
    cou = x[0]
    dat = x[1]
    
    # 优惠券null,消费日期notna，普通消费0
    if (pd.isnull(cou)) & ( pd.notnull(dat)):
        return 0
    # 优惠券null,消费日期null，不存在
    elif (pd.isnull(cou)) & ( pd.isnull(dat)):
        print('error')
        return 'error'
    # 优惠券notna,消费日期notna，优惠券消费，1
    elif (pd.notnull(cou)) & (pd.notnull(dat)):
        return 1
    # 优惠券notna,消费日期null，领券不消费，-1
    elif (pd.notnull(cou)) & (pd.isnull(dat)):
        return -1
    
dfoff.loc[:,['Coupon_id','Date']].head().apply(get_label,axis=1)

0    0
1   -1
2   -1
3   -1
4   -1
dtype: int64

In [73]:
def get_label_off(dfoff):
    """给样本打标签"""
    # 拿券，消费
    dfoff.loc[(dfoff.Coupon_id.notna()) & (dfoff.Date.notna()),'label'] = 1
    # 拿券，不消费
    dfoff.loc[(dfoff.Coupon_id.notna()) & (dfoff.Date.isna()),'label'] = -1
    #不拿券，消费
    dfoff.loc[(dfoff.Coupon_id.isna()) & (dfoff.Date.notna()),'label'] = 0
    #不拿券，不消费
    dfoff.loc[(dfoff.Coupon_id.isna()) & (dfoff.Date.isna()),'label'] = 'error'
    return dfoff

In [74]:
#打标签
dfoff = get_label_off(dfoff)

In [75]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN,-1
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,-1
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,-1
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN,-1


In [70]:
#划分子集
dfoff.label.value_counts()
#看起来，整体核销率就是，5%左右

2    977900
0    701602
1     75382
Name: label, dtype: int64

In [81]:
dfall = dfoff[dfoff.label!=-1]
dfall.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,0
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0,0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0,1
11,2029232,6459,NaN,NaN,0.0,NaN,20160626.0,0
12,2029232,6459,NaN,NaN,0.0,NaN,20160519.0,0


In [ ]:
#